## Imports and Defines

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [3]:
teams = pd.read_csv('./src/teams.csv')
players = pd.read_csv('./src/players.csv')
matches = pd.read_csv('./src/matches.csv')

## Players Dataset

In [7]:
missing_players_values = players.isnull().sum()
missing_players_percentage = (missing_players_values / players.shape[0]) * 100
missing_players_percentage.sort_values(ascending=False)

attacking_work_rate    3.733599
agility                1.474633
balance                1.474633
jumping                1.474633
sliding_tackle         1.474633
curve                  1.474633
vision                 1.474633
volleys                1.474633
aggression             0.454402
shot_power             0.454402
stamina                0.454402
strength               0.454402
long_shots             0.454402
sprint_speed           0.454402
interceptions          0.454402
reactions              0.454402
penalties              0.454402
marking                0.454402
standing_tackle        0.454402
gk_diving              0.454402
gk_handling            0.454402
gk_kicking             0.454402
gk_positioning         0.454402
positioning            0.454402
gk_reflexes            0.454402
crossing               0.454402
ball_control           0.454402
long_passing           0.454402
free_kick_accuracy     0.454402
dribbling              0.454402
short_passing          0.454402
heading_

Пропусков менее 4%. Заполним пропуски средним значением по возрасту.

## Teams Dataset

In [13]:
missing_teams_values = teams.isnull().sum()
missing_teams_percentage = (missing_teams_values / teams.shape[0]) * 100
missing_teams_percentage.sort_values(ascending=False)

team_api_id                       0.0
team_long_name                    0.0
buildUpPlayPassing                0.0
buildUpPlayPassingClass           0.0
buildUpPlayPositioningClass       0.0
buildUpPlaySpeed                  0.0
buildUpPlaySpeedClass             0.0
chanceCreationCrossing            0.0
chanceCreationCrossingClass       0.0
chanceCreationPassing             0.0
chanceCreationPassingClass        0.0
chanceCreationPositioningClass    0.0
chanceCreationShooting            0.0
chanceCreationShootingClass       0.0
date                              0.0
defenceAggression                 0.0
defenceAggressionClass            0.0
defenceDefenderLineClass          0.0
defencePressure                   0.0
defencePressureClass              0.0
defenceTeamWidth                  0.0
defenceTeamWidthClass             0.0
team_short_name                   0.0
buildUpPlayDribblingClass         0.0
dtype: float64

Удалим атрибуты, содержащие > 60% пропусков. Остальные атрибуты пропусков не имеют.

In [12]:
teams_columns_to_drop = missing_teams_percentage[missing_teams_percentage > 60].index
teams.drop(columns=teams_columns_to_drop, inplace=True)

## Matches Dataset

In [16]:
missing_matches_values = matches.isnull().sum()
missing_matches_percentage = (missing_matches_values / matches.shape[0]) * 100
missing_matches_percentage.sort_values(ascending=False)

IWD            13.314600
IWH            13.314600
IWA            13.314600
LBA            13.176027
LBD            13.176027
                 ...    
date            0.000000
stage           0.000000
season          0.000000
league_id       0.000000
league_name     0.000000
Length: 97, dtype: float64

Имеем очень много пропусков в атрибутах. Удалим атрибуты с более 30% пропусков.

In [15]:
matches_columns_to_drop = missing_matches_percentage[missing_matches_percentage > 30].index
matches.drop(columns=matches_columns_to_drop, inplace=True)